In [1]:
!cat requirements.txt | xargs -n 1 -L 1 pip install --upgrade
!conda install -c conda-forge -y ta-lib

  Using cached numpy-1.23.5.tar.gz (10.7 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [33 lines of output]
      Traceback (most recent call last):
        File "/opt/anaconda3/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 353, in <module>
          main()
        File "/opt/anaconda3/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "/opt/anaconda3/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 112, in get_requires_for_build_wheel
          backend = _build_backend()
                    ^^^^^^^^^^^^^^^^
        File "/opt/anaconda3

In [2]:
!pip install ta-lib

In [3]:
import pandas as pd
import talib

from google.cloud import bigquery
from google.cloud import bigquery_storage

In [4]:
import tidal as td

/opt/anaconda3/lib/python3.12/site-packages/pyfolio/pos.py:25: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  warnings.warn(


In [5]:
bq_client = bigquery.Client(project='dst-dev2021')
bq_storage = bigquery_storage.BigQueryReadClient()

[2024/12/20 17:42:36] google.auth._default WARNING No project ID could be determined. Consider running `gcloud config set project` or setting the GOOGLE_CLOUD_PROJECT environment variable
[2024/12/20 17:42:36] google.auth._default WARNING No project ID could be determined. Consider running `gcloud config set project` or setting the GOOGLE_CLOUD_PROJECT environment variable


In [6]:
query_quote = """
    SELECT instrument, DATETIME(datetime, 'Asia/Taipei') as datetime, open, high, low, close, volume
    FROM quant_data.`tw-stocks_1day`
    WHERE instrument in (
        SELECT instrument FROM quant_data.`tw-stocks_pool_TSE`
    ) AND datetime >= TIMESTAMP("2019-01-01", 'Asia/Taipei')
    ORDER BY instrument, datetime;
"""
quote_data = bq_client.query(query_quote).to_dataframe(bqstorage_client=bq_storage, progress_bar_type="tqdm")
quote_data = quote_data.set_index(['instrument', 'datetime'])

instruments = quote_data.index.get_level_values('instrument').unique()

new_db = {}

for instrument in instruments:
    new_db[instrument] = quote_data.loc[instrument].assign(MACD=talib.MACD(quote_data.loc[instrument].close, fastperiod=12, slowperiod=26, signalperiod=9)[2])

quote_data = pd.concat(new_db, names=["instrument"], sort=False).dropna()
quote_data

Job ID e3170067-50c4-41ca-b3d4-459b77727adc successfully executed: 100%|██████████|

/opt/anaconda3/lib/python3.12/site-packages/google/cloud/bigquery/table.py:2309: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
/opt/anaconda3/lib/python3.12/site-packages/google/cloud/bigquery/table.py:2323: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
/opt/anaconda3/lib/python3.12/site-packages/google/cloud/bigquery/table.py:2337: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)



Downloading: 100%|██████████|


open        high         low       close  \
instrument datetime                                                     
1101       2019-02-27   23.352589   23.505419   23.322022   23.474854   
           2019-03-04   23.474851   23.505417   23.230321   23.352587   
           2019-03-05   23.291456   23.413720   23.199756   23.230322   
           2019-03-06   23.230322   23.413721   23.199757   23.383154   
           2019-03-07   23.291456   23.505418   23.230322   23.230322   
...                           ...         ...         ...         ...   
9958       2024-12-13  180.000000  183.000000  176.500000  179.000000   
           2024-12-16  180.500000  181.000000  174.000000  174.000000   
           2024-12-17  176.500000  181.500000  176.500000  180.000000   
           2024-12-18  181.500000  191.500000  181.500000  188.000000   
           2024-12-19  182.500000  184.500000  172.500000  174.500000   

                           volume      MACD  
instrument datetime                          
1101       2019-02-27  17778259.0  0.018697  
           2019-03-04  16341410.0 -0.002826  
           2019-03-05  10695312.0 -0.027861  
           2019-03-06   9571283.0 -0.036165  
           2019-03-07  15178932.0 -0.052971  
...                           ...       ...  
9958       2024-12-13   1740240.0  1.327440  
           2024-12-16   2069451.0  0.905278  
           2024-12-17   2501249.0  1.028497  
           2024-12-18  10230800.0  1.612382  
           2024-12-19  11987084.0  1.078640  

[1353859 rows x 6 columns]